In [ ]:
import os
import numpy as np

# defining global variable path
image_path = "./images_training_rev1/new_images/"

#function to load folder into arrays and then it returns that same array'''
def loadImages(path):
    # Put files into lists and return them as one list of size 4
    image_files = sorted([os.path.join(path, file)
         for file in os.listdir(path) if file.endswith('.jpg')])
 
    return image_files

In [ ]:
data_img = loadImages(image_path)

In [ ]:
import cv2
img = [cv2.imread(i, cv2.IMREAD_UNCHANGED) for i in data_img]
print(img[0].shape)

In [ ]:
img = np.asarray(img)

In [ ]:
import pandas as pd
data = pd.read_csv('class1_df.csv', index_col=0)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(img, data, test_size=.2, random_state=15)
print(X_train.shape)
print(y_train.shape)

In [ ]:
from keras.applications import VGG16
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, GlobalMaxPooling2D, Conv2D, MaxPooling2D

seed = 7
np.random.seed(seed)
tf.set_random_seed(seed)

# ignore Warning
import warnings
warnings.filterwarnings("ignore")

pretrained_model =VGG16(include_top=False, input_shape=(64, 64, 3), weights='imagenet')
for layer in pretrained_model.layers:
    layer.trainable = False

model = Sequential()
for i in range(11):
    model.add(pretrained_model.layers[i])
    
pretrained_model = model

X_train_pretrained_features = pretrained_model.predict(X_train)
X_val_pretrained_features = pretrained_model.predict(X_val)

In [ ]:
model.summary()

In [ ]:
X_train_pretrained_features.shape[1:]

In [ ]:
import keras
from keras.optimizers import SGD, Adamax, Adadelta, Adam, Nadam 

seed = 7
np.random.seed(seed)
tf.set_random_seed(seed)

model = Sequential()
model.add(Conv2D(filters=8, kernel_size=(3,3),input_shape=X_train_pretrained_features.shape[1:],activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(3, activation='softmax'))

In [ ]:
loss = 'categorical_crossentropy'
LEARNING_RATE = 0.001
EPOCHS = 100

model.compile(loss=loss,
               optimizer=Adamax(lr=LEARNING_RATE), # SGD(lr=LEARNING_RATE)
               metrics=['accuracy'])

history = model.fit(X_train_pretrained_features,
                     y_train,
                     epochs=EPOCHS,
                     validation_data=(X_val_pretrained_features, y_val),
                     batch_size=X_train_pretrained_features.shape[1]
                     )

In [ ]:
model.summary()

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.title('Model performance throughout training')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
# Predict.
predictions = model.predict(X_val)
# Print our model's predictions.
print("predictions: " + str(predictions[:1]))

# Check our predictions against the ground truths.
print(y_val[:1]) 

In [ ]:
score = model.evaluate(X_val, y_val, verbose=0)
print('Val loss:', score[0])
print('Val accuracy:', score[1])

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.title('Model performance throughout training')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import accuracy_score

y_train_class = y_test.argmax(axis=1)
y_pred_class = predictions.argmax(axis=1)
conf_mat = confusion_matrix(y_train_class, y_pred_class)
plt.figure(figsize=(15,10))
sns.heatmap(conf_mat,square=True, annot=True, fmt='.0f')
plt.show()
print("accuracy :" + str((accuracy_score(y_train_class, y_pred_class).round(3))*100)+" %")